# Library of Congress Data Exploration
### University of Michigan School of Information
### SI 370

- Grace Knitter
- Jack Gioffre
- Dakota Lewis
- David Forystek


In [1]:
#>>> pip --version

In [2]:
#>>> pip install -U pip

In [3]:
#>>> pip install nltk

In [4]:
#>>> pip install gensim

In [5]:
#nltk.download()

# GIPHY HISTORICAL DATA

1. NLTK PROCESSING
    - NLTK
    - Word 2 Vec
    - Vectorization

2. DATA MANIPULATION
    - Cleaning Data
    - Normalizing dtypes

3. ENUMERATING AND REDUCTION
    - Converting the values of each coulumn into a numeric float
    - Deciding which columns to keep

4. MACHINE LEARNING & PREDICTIONS
    - Pipeline reduction
    - Accuracy Tests

5. DATA VISUALIZATIONS
    - Findings
    - Visualizations

## NLTK PROCESSING
- Using NLTK processing on the dataset to tokenize each word,
- Create a Word to Vector model using the words found in the database,
- Assign each word of a rows sentence into a distint column that corresponds to its positions,
- Convert the Columns of words into floats using the mean of the Word2Vec model

In [6]:
import pandas as pd
import numpy as np
import datetime
import nltk
import gensim

In [7]:
import warnings
warnings.filterwarnings("ignore")

In [8]:
gif_data = pd.read_csv("Gif_data.csv")

In [9]:
gif_data = gif_data.drop(['Web Archive Link','MD5 Hash'],axis = 1)

In [10]:
gif_data.head()

,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date
0,1000eGIbYHercI,82505,slipper GIF,9/21/13 3:37,1/1/70 0:00
1,1000fHsBSKSL6w,102564,swag hustling GIF,6/22/15 5:35,1/12/16 23:45
2,1000WjcUQeqOaY,48315,jennifer lawrence salute GIF,3/22/13 11:27,1/1/70 0:00
3,1003Kuq13ICXDO,104921,love happy GIF,3/7/16 22:08,0000-00-00 00:00:00
4,1004D5v8Dbe6cw,45649,visual album beyonce GIF,12/13/13 14:32,6/20/14 7:15


In [11]:
x = gif_data.dropna()

In [12]:
gif_titles = x['GIPHY Title']
tokenlist = []
for its in gif_titles:
    token = nltk.word_tokenize(its)
    tokenlist.append(token)
x['Tokens'] = tokenlist
x_sorted_by_date = x.copy()
x_sorted_by_date.head()

,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,Tokens
0,1000eGIbYHercI,82505,slipper GIF,9/21/13 3:37,1/1/70 0:00,"[slipper, GIF]"
1,1000fHsBSKSL6w,102564,swag hustling GIF,6/22/15 5:35,1/12/16 23:45,"[swag, hustling, GIF]"
2,1000WjcUQeqOaY,48315,jennifer lawrence salute GIF,3/22/13 11:27,1/1/70 0:00,"[jennifer, lawrence, salute, GIF]"
3,1003Kuq13ICXDO,104921,love happy GIF,3/7/16 22:08,0000-00-00 00:00:00,"[love, happy, GIF]"
4,1004D5v8Dbe6cw,45649,visual album beyonce GIF,12/13/13 14:32,6/20/14 7:15,"[visual, album, beyonce, GIF]"


In [13]:
lstframe = pd.DataFrame(x_sorted_by_date['Tokens'].tolist())
y = list(lstframe.columns)

x_sorted_by_date[y] = lstframe
x_sorted_by_date = x_sorted_by_date.sort_values(by='Import Date')

In [14]:

x_sorted_by_date.head()

,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,Tokens,0,1,2,3,...,12,13,14,15,16,17,18,19,20,21
2377,12r4pHjvAOv48o,107392,pokemon GIF,1/1/14 20:13,1/1/70 0:00,"[pokemon, GIF]",the,nanny,grandma,GIF,...,None,None,None,None,None,None,None,None,None,None
1161,11dgYWwHnEFomQ,83047,lunch eating GIF,1/1/14 21:23,6/20/14 20:15,"[lunch, eating, GIF]",kitten,fear,GIF,None,...,None,None,None,None,None,None,None,None,None,None
7400,3eYcUcetwdtp6,53518,golden girls GIF,1/1/14 7:24,1/1/70 0:00,"[golden, girls, GIF]",robot,haters,gon,na,...,None,None,None,None,None,None,None,None,None,None
4254,1LxTmOgJFveo0,44254,once upon a time if somethin g happens to u im...,1/1/14 8:12,1/1/70 0:00,"[once, upon, a, time, if, somethin, g, happens...",behind,the,scenes,chuck,...,None,None,None,None,None,None,None,None,None,None
6758,2pcA7x3SXdiZG,107138,hear me roar game of thrones GIF,1/1/15 20:05,1/1/70 0:00,"[hear, me, roar, game, of, thrones, GIF]",new,years,cat,GIF,...,None,None,None,None,None,None,None,None,None,None


In [15]:
from gensim.models.word2vec import Word2Vec
tokens = x_sorted_by_date
model = Word2Vec(sentences=tokens['Tokens'], window=5, min_count=1, workers=4)
model.save("word2vec.model")

In [16]:
token_columns= list(x_sorted_by_date.columns[6:-6])
token_columns

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [17]:
def score_func(db):
    if db in model.wv.vocab:
        return model[db].mean()
    else:
        return 0.0
#     for x in db.str:
#             b = model[db]
#             return b.mean()
#         else:
#             return 0.0
for h in token_columns:
    x_sorted_by_date[h] = x_sorted_by_date[h].apply(score_func)

In [18]:
X_Data = x_sorted_by_date.fillna(0.0)
X_Data.head()

,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,Tokens,0,1,2,3,...,12,13,14,15,16,17,18,19,20,21
2377,12r4pHjvAOv48o,107392,pokemon GIF,1/1/14 20:13,1/1/70 0:00,"[pokemon, GIF]",0.001870,0.000063,-0.000051,0.002436,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
1161,11dgYWwHnEFomQ,83047,lunch eating GIF,1/1/14 21:23,6/20/14 20:15,"[lunch, eating, GIF]",0.000471,-0.000061,0.002436,0.000000,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
7400,3eYcUcetwdtp6,53518,golden girls GIF,1/1/14 7:24,1/1/70 0:00,"[golden, girls, GIF]",0.000158,0.000120,0.000809,0.000084,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
4254,1LxTmOgJFveo0,44254,once upon a time if somethin g happens to u im...,1/1/14 8:12,1/1/70 0:00,"[once, upon, a, time, if, somethin, g, happens...",0.000211,0.001870,-0.000133,0.000198,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0
6758,2pcA7x3SXdiZG,107138,hear me roar game of thrones GIF,1/1/15 20:05,1/1/70 0:00,"[hear, me, roar, game, of, thrones, GIF]",0.000730,0.000264,0.000410,0.002436,...,0.0,0.0,0.0,0.0,0,0,0,0,0,0


## DATA MANIPULATION
- Cleaning Data
- Normalizing dtypes
- Convert the string dates into date times
- Assign the Year, Day, and Month that a GIPHY was imported and/or started trending to separate columns
- Run correlation tests to see if any of the columns correlate to weather or not a GIPHY trends


In [19]:
X_Data['Trending Date'] = X_Data['Trending Date'].str.replace("1/1/70 0:00",'0000-00-00 00:00:00')
X_Data['Import Date'] = X_Data['Import Date'].str.replace("1/1/70 0:00",'0000-00-00 00:00:00')


In [20]:
X_Data['trended'] = X_Data['Trending Date'] != '0000-00-00 00:00:00'
X_Data['imported'] = X_Data['Import Date'] != '0000-00-00 00:00:00'

In [21]:
X_Data['Import Date'].value_counts()

0000-00-00 00:00:00    129
4/28/16 0:33             6
5/6/16 20:05             5
8/9/16 17:22             5
2/23/16 15:40            5
                      ... 
8/14/15 14:50            1
3/1/16 17:10             1
7/1/15 1:19              1
7/18/16 2:47             1
8/11/16 15:08            1
Name: Import Date, Length: 9869, dtype: int64

In [22]:
X_Data.loc[(X_Data['trended'] == True) & (X_Data['imported'] == True)].dropna().head()

,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,Tokens,0,1,2,3,...,14,15,16,17,18,19,20,21,trended,imported
1161,11dgYWwHnEFomQ,83047,lunch eating GIF,1/1/14 21:23,6/20/14 20:15,"[lunch, eating, GIF]",0.000471,-0.000061,0.002436,0.000000,...,0.0,0.0,0,0,0,0,0,0,True,True
8148,3o6LXuw8NdVed4IMx2,97781,ryan seacrest selfie GIF by New Year's Rockin'...,1/1/16 1:17,1/1/16 1:24,"[ryan, seacrest, selfie, GIF, by, New, Year, '...",0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0,0,0,0,0,0,True,True
3713,14qVHqPNQNDsPu,58239,confused jeremy renner GIF,1/10/14 19:52,6/1/15 23:53,"[confused, jeremy, renner, GIF]",0.000505,0.001183,0.000269,0.002436,...,0.0,0.0,0,0,0,0,0,0,True,True
5690,26gGx4jp8m6Xk4IRW,415837,yellow pop GIF by DLGNCE,1/10/16 21:37,8/10/16 6:15,"[yellow, pop, GIF, by, DLGNCE]",0.000320,-0.000002,0.000551,0.002436,...,0.0,0.0,0,0,0,0,0,0,True,True
329,10gHObFunIQ6nm,75931,adventure time GIF,1/10/16 2:42,4/26/16 14:15,"[adventure, time, GIF]",0.000474,0.002286,0.001589,0.000581,...,0.0,0.0,0,0,0,0,0,0,True,True


In [23]:
Trends = X_Data.loc[(X_Data['trended'] == True) & (X_Data['imported'] == True)].dropna().copy()


In [24]:
len(Trends)

4620

In [25]:
Trends['Trending Date']= pd.to_datetime(Trends['Trending Date'])
Trends['Import Date'] = pd.to_datetime(Trends['Import Date'])

In [26]:
Trends['Time to trend'] = Trends['Trending Date'] - Trends['Import Date']


In [27]:
gifs_trended = Trends#.loc[Trends['Time to trend']>='0 days 00:00:00']

In [28]:
gifs_trended.sort_values('Time to trend').head()

,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,Tokens,0,1,2,3,...,15,16,17,18,19,20,21,trended,imported,Time to trend
3161,13rccieMQwGwyk,185198,GIF by kidmograph,2014-06-19 16:37:00,2014-03-22 18:27:00,"[GIF, by, kidmograph]",0.000617,0.000044,0.000058,0.000783,...,0.0,0,0,0,0,0,0,True,True,-89 days +01:50:00
3806,14xRFcorZuVJ0k,5295,yasiel puig mlb GIF,2014-06-19 15:09:00,2014-03-25 09:26:00,"[yasiel, puig, mlb, GIF]",-0.000018,0.000849,0.000434,0.002436,...,0.0,0,0,0,0,0,0,True,True,-87 days +18:17:00
1660,11utShiPFzSR8s,252821,ae GIF by kidmograph,2014-06-19 14:14:00,2014-04-11 14:25:00,"[ae, GIF, by, kidmograph]",-0.000212,-0.000367,-0.000124,0.001870,...,0.0,0,0,0,0,0,0,True,True,-69 days +00:11:00
10277,3o85xC2d2e90B5sUs8,111793,fifth harmony surfboard GIF by FOX Teen Choice,2015-08-17 16:31:00,2015-08-17 01:54:00,"[fifth, harmony, surfboard, GIF, by, FOX, Teen...",0.000783,0.001427,0.000466,0.002436,...,0.0,0,0,0,0,0,0,True,True,-1 days +09:23:00
8717,3o6Zt9jaIlgVHWUZ1e,64248,burnt stewie griffin GIF by Family Guy,2016-07-12 06:33:00,2016-07-11 22:45:00,"[burnt, stewie, griffin, GIF, by, Family, Guy]",-0.000825,-0.004256,0.000207,0.002436,...,0.0,0,0,0,0,0,0,True,True,-1 days +16:12:00


In [29]:
gifs_trended.reset_index(inplace=True)

In [30]:
def convert_timedelta(duration):
    days, seconds = duration.days ,duration.seconds
    hours = days * 24 + seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = (seconds % 60)
    return str(hours)+':'+ str(minutes)+':'+str(seconds)
gifs_trended['Time'] = pd.to_timedelta(gifs_trended['Time to trend']).apply(convert_timedelta)

In [31]:
pd.to_timedelta(gifs_trended['Time to trend'])

0      169 days 22:52:00
1        0 days 00:07:00
2      507 days 04:01:00
3      212 days 08:38:00
4      107 days 11:33:00
              ...       
4615   743 days 06:35:00
4616   217 days 19:13:00
4617     6 days 03:39:00
4618     0 days 00:20:00
4619   204 days 10:50:00
Name: Time to trend, Length: 4620, dtype: timedelta64[ns]

In [32]:
gifs_trended.sort_values('Time').head()

,index,GIF ID,File Size (In Bytes),GIPHY Title,Import Date,Trending Date,Tokens,0,1,2,...,16,17,18,19,20,21,trended,imported,Time to trend,Time
4144,10277,3o85xC2d2e90B5sUs8,111793,fifth harmony surfboard GIF by FOX Teen Choice,2015-08-17 16:31:00,2015-08-17 01:54:00,"[fifth, harmony, surfboard, GIF, by, FOX, Teen...",0.000783,0.001427,0.000466,...,0,0,0,0,0,0,True,True,-1 days +09:23:00,-15:23:0
3019,1660,11utShiPFzSR8s,252821,ae GIF by kidmograph,2014-06-19 14:14:00,2014-04-11 14:25:00,"[ae, GIF, by, kidmograph]",-0.000212,-0.000367,-0.000124,...,0,0,0,0,0,0,True,True,-69 days +00:11:00,-1656:11:0
3523,8703,3o6Zt709aIBCEMQGVG,126629,euro 2016 portugal GIF,2016-07-11 00:28:00,2016-07-10 23:28:00,"[euro, 2016, portugal, GIF]",0.000704,0.000724,0.000201,...,0,0,0,0,0,0,True,True,-1 days +23:00:00,-1:0:0
3935,9639,3o7TKAwnruRFfywQ0M,102676,andrew cuomo backbone of society GIF by Electi...,2016-07-29 00:31:00,2016-07-28 23:45:00,"[andrew, cuomo, backbone, of, society, GIF, by...",0.001134,0.002448,0.002286,...,0,0,0,0,0,0,True,True,-1 days +23:14:00,-1:14:0
2701,9500,3o7qE1dlOTVTCNpxfi,126716,music video dancing GIF,2016-05-06 19:03:00,2016-05-06 18:24:00,"[music, video, dancing, GIF]",0.000000,0.000000,0.000000,...,0,0,0,0,0,0,True,True,-1 days +23:21:00,-1:21:0


In [33]:
x = gifs_trended
x.set_index('Import Date',inplace=True)
x.head()

,index,GIF ID,File Size (In Bytes),GIPHY Title,Trending Date,Tokens,0,1,2,3,...,16,17,18,19,20,21,trended,imported,Time to trend,Time
Import Date,,,,,,,,,,,,,,,,,,,,,
2014-01-01 21:23:00,1161,11dgYWwHnEFomQ,83047,lunch eating GIF,2014-06-20 20:15:00,"[lunch, eating, GIF]",0.000471,-0.000061,0.002436,0.000000,...,0,0,0,0,0,0,True,True,169 days 22:52:00,4078:52:0
2016-01-01 01:17:00,8148,3o6LXuw8NdVed4IMx2,97781,ryan seacrest selfie GIF by New Year's Rockin'...,2016-01-01 01:24:00,"[ryan, seacrest, selfie, GIF, by, New, Year, '...",0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,True,True,0 days 00:07:00,0:7:0
2014-01-10 19:52:00,3713,14qVHqPNQNDsPu,58239,confused jeremy renner GIF,2015-06-01 23:53:00,"[confused, jeremy, renner, GIF]",0.000505,0.001183,0.000269,0.002436,...,0,0,0,0,0,0,True,True,507 days 04:01:00,12172:1:0
2016-01-10 21:37:00,5690,26gGx4jp8m6Xk4IRW,415837,yellow pop GIF by DLGNCE,2016-08-10 06:15:00,"[yellow, pop, GIF, by, DLGNCE]",0.000320,-0.000002,0.000551,0.002436,...,0,0,0,0,0,0,True,True,212 days 08:38:00,5096:38:0
2016-01-10 02:42:00,329,10gHObFunIQ6nm,75931,adventure time GIF,2016-04-26 14:15:00,"[adventure, time, GIF]",0.000474,0.002286,0.001589,0.000581,...,0,0,0,0,0,0,True,True,107 days 11:33:00,2579:33:0


In [34]:
#x['Time'].str.split(':')[0:-1][1][0]
hrs = []
mins = []
for time in x['Time'].str.split(':'):
    hrs.append(int(time[0]))
    mins.append(int(time[1]))
x['Mins']=mins
x['Hours']=hrs

In [35]:
x['Hours'] = x['Hours'] + x['Mins']/60

In [36]:
x.drop(['Time','Mins'],axis=1,inplace=True)

In [37]:
visual = x.copy()
visual['log'] = visual['Hours'].apply(np.log)
#visual

In [38]:
visual['Hours'].describe()
visual.set_index('GIPHY Title',inplace=True)

In [39]:
# visual['Hours'].plot(figsize=[100,50])


In [40]:
# visual['Hours'].apply(np.log).plot(figsize=[100,50])


In [41]:
import seaborn as sns

In [42]:
visual.columns

Index([               'index',               'GIF ID', 'File Size (In Bytes)',
              'Trending Date',               'Tokens',                      0,
                            1,                      2,                      3,
                            4,                      5,                      6,
                            7,                      8,                      9,
                           10,                     11,                     12,
                           13,                     14,                     15,
                           16,                     17,                     18,
                           19,                     20,                     21,
                    'trended',             'imported',        'Time to trend',
                      'Hours',                  'log'],
      dtype='object')

In [43]:
# sns.lineplot(y=visual['Hours'],x=visual.index)

In [44]:
x['import']=x.index
x = x.reset_index()

## 3. ENUMERATING AND REDUCTION
- Converting the values of each column into a numeric float
- Removing Columns that couldn't be enumerated
- Converting and replacing necessary values and columns
- Assign values to NaNs
- Reducing the Data into 2 Datasets
        - GIF data with NLTK
        - GIF data with no NLTK


In [45]:
x2 = x.copy()

In [46]:
Numer_db = x2.copy()

In [47]:
Numer_db.head()

,Import Date,index,GIF ID,File Size (In Bytes),GIPHY Title,Trending Date,Tokens,0,1,2,...,17,18,19,20,21,trended,imported,Time to trend,Hours,import
0,2014-01-01 21:23:00,1161,11dgYWwHnEFomQ,83047,lunch eating GIF,2014-06-20 20:15:00,"[lunch, eating, GIF]",0.000471,-0.000061,0.002436,...,0,0,0,0,0,True,True,169 days 22:52:00,4078.866667,2014-01-01 21:23:00
1,2016-01-01 01:17:00,8148,3o6LXuw8NdVed4IMx2,97781,ryan seacrest selfie GIF by New Year's Rockin'...,2016-01-01 01:24:00,"[ryan, seacrest, selfie, GIF, by, New, Year, '...",0.000000,0.000000,0.000000,...,0,0,0,0,0,True,True,0 days 00:07:00,0.116667,2016-01-01 01:17:00
2,2014-01-10 19:52:00,3713,14qVHqPNQNDsPu,58239,confused jeremy renner GIF,2015-06-01 23:53:00,"[confused, jeremy, renner, GIF]",0.000505,0.001183,0.000269,...,0,0,0,0,0,True,True,507 days 04:01:00,12172.016667,2014-01-10 19:52:00
3,2016-01-10 21:37:00,5690,26gGx4jp8m6Xk4IRW,415837,yellow pop GIF by DLGNCE,2016-08-10 06:15:00,"[yellow, pop, GIF, by, DLGNCE]",0.000320,-0.000002,0.000551,...,0,0,0,0,0,True,True,212 days 08:38:00,5096.633333,2016-01-10 21:37:00
4,2016-01-10 02:42:00,329,10gHObFunIQ6nm,75931,adventure time GIF,2016-04-26 14:15:00,"[adventure, time, GIF]",0.000474,0.002286,0.001589,...,0,0,0,0,0,True,True,107 days 11:33:00,2579.550000,2016-01-10 02:42:00


In [48]:
Nums = Numer_db.drop(['trended','imported','Tokens','Time to trend','GIPHY Title','File Size (In Bytes)','Trending Date','Import Date',16,17,18,19,20,21],axis=1)

In [49]:
#Numer_db
x3 = Nums.merge(X_Data.drop([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21],axis=1),how='outer',on='GIF ID')

In [50]:
BaseML_Gif_DB = x3.fillna(0.0)

In [51]:
BaseML_Gif_DB.loc[BaseML_Gif_DB['Hours']== (pd.Timedelta(seconds=0))]= BaseML_Gif_DB.loc[BaseML_Gif_DB['Hours']== (pd.Timedelta(seconds=0))].replace((pd.Timedelta(seconds=0)),0.0)


In [52]:
def yr(row):
    if row == pd.Timestamp('1970-01-01 00:00:00'):
        return 0.0
    else:
        return row.normalize().year
def month(row):
    if row == pd.Timestamp('1970-01-01 00:00:00'):
        return 0.0
    else:
        return row.normalize().month
def days(row):
    if row == pd.Timestamp('1970-01-01 00:00:00'):
        return 0.0
    else:
        return row.normalize().day
def logs(row):
    if row != 0.000000:
        return np.log(row)
    else:
        return 0.0 

In [53]:
BaseML_Gif_DB['Trending Date'] = BaseML_Gif_DB['Trending Date'].replace('0000-00-00 00:00:00',pd.Timestamp("1/1/70 0:00"))
BaseML_Gif_DB['Import Date'] = BaseML_Gif_DB['Import Date'].replace('0000-00-00 00:00:00',pd.Timestamp("1/1/70 0:00"))


In [54]:
BaseML_Gif_DB['Trending Date'].value_counts()

1970-01-01 00:00:00    5796
4/27/16 21:42             5
6/9/16 2:22               4
8/1/16 2:41               3
7/12/16 1:45              3
                       ... 
11/18/14 18:22            1
10/5/15 19:24             1
3/3/16 16:49              1
5/11/16 2:36              1
1/20/17 10:45             1
Name: Trending Date, Length: 4611, dtype: int64

In [55]:
BaseML_Gif_DB['Hours'] = pd.to_numeric(BaseML_Gif_DB['Hours'])
BaseML_Gif_DB['log'] = BaseML_Gif_DB['Hours'].apply(logs)
BaseML_Gif_DB['log'] = pd.to_numeric(BaseML_Gif_DB['log'])

In [56]:
BaseML_Gif_DB_3 = BaseML_Gif_DB.loc[BaseML_Gif_DB['Import Date']!= 0.0].copy()

In [57]:
BaseML_Gif_DB_3['Import Date'] = pd.to_datetime(BaseML_Gif_DB_3['Import Date'])
BaseML_Gif_DB_3['Trending Date'] = pd.to_datetime(BaseML_Gif_DB_3['Trending Date'])

In [58]:
BaseML_Gif_DB_3['Days'] = BaseML_Gif_DB_3['Import Date'].apply(days)
BaseML_Gif_DB_3['Month'] = BaseML_Gif_DB_3['Import Date'].apply(month)
BaseML_Gif_DB_3['Year'] = BaseML_Gif_DB_3['Import Date'].apply(yr)


BaseML_Gif_DB_3['TrndDays'] = BaseML_Gif_DB_3['Trending Date'].apply(days)
BaseML_Gif_DB_3['TrndYear'] = BaseML_Gif_DB_3['Trending Date'].apply(yr)
BaseML_Gif_DB_3['TrndMonth'] = BaseML_Gif_DB_3['Trending Date'].apply(month)

## 4. MACHINE LEARNING & PREDICTIONS
- Duplicate databases to make non-destructive edits
- Assign "Actual Trended" column in Databases to a boolean
- Split the Data into a training set and a testing set with X values and Y labels
- Create a pipeline from the values using a predictive classifier
- Use the generated prepared data to prepare to run accuracy tests on:
    - Decision Tree
    - Random Forest
    - Neural Net
    - AdaBoost
    - Naive Bayes
- We chose these tests because using the others took an indefinite period of time and caused crashes.
- Predict the accuracy of the database with NLTK vectorization and manipulation vs the database with no NLTK values and using the base datasets numerical values to predict
- Recorded Results


In [59]:
Reduced_df = BaseML_Gif_DB_3.drop(['index','TrndYear','TrndMonth','TrndDays','Trending Date'],axis=1)

In [60]:
Reduced_df.rename({0:'0',1:'1',2:'2',3:'3',4:'4',5:'5',6:'6',7:'7',8:'8',9:'9',10:'10',11:'11',12:'12',13:'13',14:'14',15:'15'},axis=1,inplace=True)

In [61]:
Reduced_df.corr()["Hours"].sort_values()

File Size (In Bytes)   -0.046308
12                     -0.013520
11                     -0.006273
13                     -0.001501
9                       0.005451
8                       0.005691
10                      0.006791
Month                   0.007969
14                      0.009526
15                      0.017131
7                       0.020261
Days                    0.030820
Year                    0.041126
imported                0.042509
6                       0.044421
5                       0.077978
4                       0.102215
0                       0.140543
1                       0.147969
3                       0.233908
2                       0.245308
trended                 0.424053
log                     0.749489
Hours                   1.000000
Name: Hours, dtype: float64

In [62]:
Reduced_df["Actual Trended"] = Reduced_df['Hours']!= 0.0

In [63]:
Reduced_df['Days'].unique()

array([ 1., 10., 11., 12., 13., 14., 15., 16., 17., 18., 19.,  2., 20.,
       21., 22., 23., 24., 25., 26., 27., 28., 29.,  3., 30., 31.,  4.,
        5.,  6.,  7.,  8.,  9.,  0.])

In [64]:
Reduced_df['File Size (In Bytes)'] = Reduced_df['File Size (In Bytes)'].astype(dtype= 'float64')
Reduced_df['Year'] = Reduced_df['Year'].astype(dtype= 'float64')
Reduced_df['Days'] = Reduced_df['Days'].astype(dtype= 'float64')
Reduced_df['Month'] = Reduced_df['Month'].astype(dtype= 'float64')

In [65]:
Reduced_df.replace({True:1,False:-1, 0:-1},inplace=True)

In [66]:
#data_no_nltk['Actual Trended'] = data_no_nltk['Actual Trended'].astype(float)


In [67]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier


In [68]:
data = Reduced_df.drop(['GIF ID','Hours','import','Import Date','GIPHY Title','Tokens','imported','trended','log'],1)

In [69]:
from sklearn.model_selection import train_test_split
# "target" contains the column name of the classification labels
target = "Actual Trended"

X = data.drop(target,1)
y = data[target]

seed = 42
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [70]:
X_train.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'File Size (In Bytes)', 'Days', 'Month', 'Year'],
      dtype='object')

In [71]:
y_train

7588   -1
8117   -1
2682    1
4871   -1
4076    1
       ..
5734   -1
5191   -1
5390   -1
860     1
7270   -1
Name: Actual Trended, Length: 7008, dtype: int64

In [72]:
X_train.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', 'File Size (In Bytes)', 'Days', 'Month', 'Year'],
      dtype='object')

In [73]:
num_attribs = data.select_dtypes(include=[np.number])
cat_attribs = data.select_dtypes(exclude=[np.number])

model = RandomForestClassifier(
    random_state=42, 
    criterion='entropy',
    max_depth=20, 
    min_samples_split=4)

model.fit(X_train,y_train)
y_pred = model.predict(X_train)
full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), y_pred),
        #("cat", OneHotEncoder(), cat_attribs),
    ])


train_X_prepared = full_pipeline.fit_transform(X_train)
test_X_prepared = full_pipeline.fit_transform(X_test)


In [74]:
names = ["Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes"]

classifiers = [
    DecisionTreeClassifier(max_depth=7),
    RandomForestClassifier(max_depth=7, n_estimators=25, max_features=20),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    ]

### PREDICTIONS : GIPHY DATA WITH NLTK vectorization and  Manipulation

In [75]:
X_train = train_X_prepared
X_test = test_X_prepared
NLTK_Results=[]
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # evaluate predictions
    accuracy = clf.score(X_test, y_test)
    NLTK_Results.append((name,accuracy * 100.0))
    print("%s Accuracy: %.2f%%" % (name,accuracy * 100.0))

Decision Tree Accuracy: 93.77%
Random Forest Accuracy: 93.77%
Neural Net Accuracy: 93.77%
AdaBoost Accuracy: 93.77%
Naive Bayes Accuracy: 67.01%


### MACHIENE LEARNING PREDICTIONS
##### GIPHY DATA WITH NO NLTK

In [76]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,File Size (In Bytes),Days,Month,Year,Actual Trended
0,0.000471,-0.000061,0.002436,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,83047.0,1.0,1.0,2014.0,1
1,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,97781.0,1.0,1.0,2016.0,1
2,0.000505,0.001183,0.000269,0.002436,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,58239.0,10.0,1.0,2014.0,1
3,0.000320,-0.000002,0.000551,0.002436,0.002059,-0.000239,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,415837.0,10.0,1.0,2016.0,1
4,0.000474,0.002286,0.001589,0.000581,0.002436,-1.000000,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,75931.0,10.0,1.0,2016.0,1


In [77]:
data_no_nltk = data.drop(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12','13', '14', '15'],1).copy()
data_no_nltk.head()

,File Size (In Bytes),Days,Month,Year,Actual Trended
0,83047.0,1.0,1.0,2014.0,1
1,97781.0,1.0,1.0,2016.0,1
2,58239.0,10.0,1.0,2014.0,1
3,415837.0,10.0,1.0,2016.0,1
4,75931.0,10.0,1.0,2016.0,1


In [78]:
from sklearn.model_selection import train_test_split
# "target" contains the column name of the classification labels
target = "Actual Trended"

X = data_no_nltk.drop(target,1)
y = data_no_nltk[target]

seed = 42
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [79]:
num_attribs = data_no_nltk.select_dtypes(include=[np.number])
cat_attribs = data_no_nltk.select_dtypes(exclude=[np.number])

model = RandomForestClassifier(
    random_state=42, 
    criterion='entropy',
    max_depth=20, 
    min_samples_split=4)

model.fit(X_train,y_train)
y_pred = model.predict(X_train)
full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), y_pred),
        #("cat", OneHotEncoder(), cat_attribs),
    ])


train_X_prepared = full_pipeline.fit_transform(X_train)
test_X_prepared = full_pipeline.fit_transform(X_test)


### PREDICTIONS : GIPHY DATA WITH NO NLTK vectorization and  Manipulation

In [80]:
X_train = train_X_prepared
X_test = test_X_prepared

No_NLTK_Results = []
for name, clf in zip(names, classifiers):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # evaluate predictions
    accuracy = clf.score(X_test, y_test)
    No_NLTK_Results.append((name,accuracy * 100.0))
    print("%s Accuracy: %.2f%%" % (name,accuracy * 100.0))

Decision Tree Accuracy: 54.82%
Random Forest Accuracy: 54.27%
Neural Net Accuracy: 52.56%
AdaBoost Accuracy: 55.43%
Naive Bayes Accuracy: 51.55%


In [81]:
ndd

NameError: name 'ndd' is not defined

In [ ]:
Results_DB.T

In [ ]:
n = pd.DataFrame(No_NLTK_Results)
n[[2,3]] = pd.DataFrame(NLTK_Results)
Results_DB = n.drop(2,1).rename(columns={1: "No NLTK", 3: "NLTK"}).set_index(0).T

In [ ]:
Results_DB.plot(figsize = (10,10))

In [ ]:
Results_DB2 = Results_DB.reset_index()

In [ ]:
Results_DB.T.reset_index().plot()

In [ ]:
Decision Tree Accuracy: 93.77%
Random Forest Accuracy: 93.77%
Neural Net Accuracy: 63.51%
Naive Bayes Accuracy: 93.77%

## 5. DATA VISUALIZATIONS